In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("../bases/rome_tension.csv")
data = data[['rome5','BE_id','T']]
data = data.pivot(index='BE_id',columns='rome5',values='T')

In [4]:
run rome.py

In [5]:
def count_by(df,var):
    nb = df.groupby([var],group_keys=False).size()
    nb = pd.DataFrame({var:nb.index,'nb_'+var:nb.values})
    nb = pd.merge(df,nb,on=var,how='left')
    return nb

In [6]:
tree = pd.read_csv("../bases/fap_pcs_rome.csv")
tree.columns = 'fap','fap_lab','pcs','pcs_lab','rome','qual','rome_lab'
tree = tree.filter(items=['fap','fap_lab','rome','rome_lab'])
tree['fap']= tree['fap'].fillna(method='ffill')
tree['fap_lab']= tree['fap_lab'].fillna(method='ffill')
tree = tree.dropna(subset=['rome'],axis=0)
tree = count_by(tree,'rome')

### Placing corr before competence

In [6]:
multiple = tree.query('nb_rome > 1').rome.tolist()
tree['best_faps'] = tree['fap']
tree['method'] = ''
tree['n_branch'] = ''
tree['n_common'] = ''
tree['corr'] = np.nan
tree['criterium'] = ''
for rome in multiple:
    faps = tree.loc[tree.rome == rome,'fap'].tolist()
    n_branch, n_common = [0], [0]
    fap_branch, fap_corr, fap_common = '', '', ''
    tie_branch, tie_common = False, False
    sig = 0
    for fap in faps: 
        group = tree.loc[tree.fap == fap,'rome'].tolist()
        if len(group) > 1: group.remove(rome)
        corr = np.mean([data[rome].corr(data[x]) for x in group])
        branch = sum([rome[0] == x[0] for x in group])
        common = sum([x in graph[rome] for x in group])
        if corr > sig:
            sig = corr 
            fap_corr = fap
        if branch == n_branch: tie_branch = True
        if common == n_common: tie_common = True
        if branch > n_branch[-1]:
            n_branch.append(branch) 
            fap_branch = fap
            if n_branch[-2] == branch: tie = True
            else: tie_branch = False
        if common > n_common[-1]:
            n_common.append(common) 
            fap_common = fap
            if n_common[-2] == common: tie = True
            else: tie_common = False
    if sig > 0:
        best_fap = fap_corr
        method = 'corr'
        t = sig
    else:
        if n_common[-1] > 0 and tie_common == False: 
            method = 'connex'
            best_fap = fap_common
            t = n_common[-1]
        else:
            if n_branch[-1] > 0 and tie_branch == False:
                method = 'branch'
                best_fap = fap_branch
                t = n_branch[-1]
            else: 
                method = 'failed'
                best_fap = ''
                t = ''
    tree.loc[tree.rome == rome,'best_faps'] = best_fap
    tree.loc[tree.rome == rome,'method'] = method
    tree.loc[tree.rome == rome,'n_common'] = n_common[-1]
    tree.loc[tree.rome == rome,'n_branch'] = n_branch[-1]
    tree.loc[tree.rome == rome,'corr'] = sig
    tree.loc[tree.rome == rome,'criterium'] = t


In [7]:
tree.loc[tree.method == 'failed'].sort_values(by='rome')

,fap,fap_lab,rome,rome_lab,nb_rome,best_faps,method,n_branch,n_common,corr,criterium
252,F2Z20,Ouvriers non qualifiés du travail du bois et d...,B1806,Tapisserie - décoration en ameublement,2,,failed,0,0,0.0,
258,F3Z40,Artisans du travail du bois et de l'ameublemen...,B1806,Tapisserie - décoration en ameublement,2,,failed,0,0,0.0,
171,E0Z22,"Ouvriers non qualifiés en métallurgie, verre, ...",F1706,Préfabrication en béton industriel,2,,failed,0,0,0.0,
191,E1Z43,"Autres ouvriers qualifiés en verre, céramique,...",F1706,Préfabrication en béton industriel,2,,failed,0,0,0.0,


##### Manual choices 

In [8]:
tree.loc[tree.rome == 'B1806','best_faps'] = 'F3Z40'
tree.loc[tree.rome == 'F1706','best_faps'] = 'E0Z22'

In [9]:
tree = tree.loc[tree.fap == tree.best_faps].reset_index(drop=True)
tree = tree.filter(items=['fap','fap_lab','rome','rome_lab','method','criterium'])

Attention: le rome 'L1510' n'est pas dans la table de passage FAP/ROME qui date de 2009 

In [10]:
tree.loc[531] = ['U1Z80', 'Professionnels des spectacles', 'L1510', "Films d'animation et effets spéciaux", '', '']

In [16]:
tree.loc[tree.rome == 'N4402']

,fap,fap_lab,rome,rome_lab,method,criterium
220,J3Z44,Conducteurs sur rails et d'engins de traction,N4402,Exploitation et manœuvre des remontées mécaniques,,


In [11]:
tree.to_csv("../bases/fap_rome_correlation.csv",header=True,index=False)

In [3]:
directed_clusters = pd.read_csv('../bases/directed_clusters.csv',header=None,names=['dir_clus','rome'])

In [12]:
tree = pd.merge(tree,directed_clusters,how='left',on='rome')
tree.dir_clus = tree.dir_clus.apply(lambda x: x[1:])
tree = tree.filter(items=['fap','dir_clus','fap_lab','rome','rome_lab']).sort_values(by=['fap','dir_clus','rome'])

In [126]:
#undirected_clusters = pd.read_csv('../bases/undirected_clusters.csv',header=None,names=['undir_clus','rome'])
#tree = pd.merge(tree,undirected_clusters,how='left',on='rome')
#tree.undir_clus = tree.undir_clus.apply(lambda x: x[1:])
#tree = tree.filter(items=['fap','dir_clus','undir_clus','fap_lab','rome','rome_lab']).sort_values(by=['fap','dir_clus','undir_clus','rome'])


In [127]:
tree.to_csv("../bases/fap_clusters.csv",header=True,index=False)

In [27]:
faps = tree.fap.unique().tolist()
fap_groups = {i:tree.loc[tree.fap == fap, 'rome'].tolist() for i,fap in enumerate(faps)}
clusters = tree.dir_clus.unique().tolist()
cluster_groups = {i:tree.loc[tree.dir_clus == clus, 'rome'].tolist() for i, clus in enumerate(clusters)}

fap_graph = {}
for i,group in fap_groups.items(): 
    fap_graph[i] = []
    for rome in group:
        fap_graph[i] += [j for j, fap in fap_groups.items() if list(set(fap) & set(graph[rome])) != [] and j != i ]
    fap_graph[i] = list(set(fap_graph[i]))

#### Redefine graph at the upper agregation level

In [11]:
clusters = directed_clusters.dir_clus.unique().tolist()
cluster_groups = {i+1:directed_clusters.loc[directed_clusters.dir_clus == clus, 'rome'].tolist() for i, clus in enumerate(clusters)}

In [14]:
clustered_graph = {}
for i,group in cluster_groups.items(): 
    clustered_graph[i] = []
    for rome in group:
        clustered_graph[i] += [j for j, clus in cluster_groups.items() if list(set(clus) & set(graph[rome])) != [] and j != i ]
    clustered_graph[i] = list(set(clustered_graph[i]))

In [25]:
purged_graph = {}
for rome, romes in graph.items():
    g = directed_clusters.loc[directed_clusters.rome == rome,'dir_clus'].values.tolist()[0]
    group = directed_clusters.loc[directed_clusters.dir_clus == g,'rome'].values.tolist()
    purged_graph[rome] = [x for x in romes if x not in group]

In [32]:
import json
with open('purged_graph', 'w') as f:
    json.dump(purged_graph, f)

In [35]:
directed_clusters

,dir_clus,rome
0,g1,E1106
1,g1,E1108
2,g2,E1107
3,g2,E1103
4,g3,E1104
5,g3,M1806
6,g3,M1805
7,g3,M1804
8,g3,M1801
9,g3,H1102
